In [ ]:
# Basic libraries
import numpy as np
import tensorflow as tf
# LSTM-autoencoder
from LSTMAutoencoder import *
from pandas import read_csv
from sklearn.preprocessing import MinMaxScaler


In [ ]:
tf.reset_default_graph()
tf.set_random_seed(2016)
np.random.seed(2016)

In [ ]:
# Constants
batch_num = 120
hidden_num = 15
step_num = 15
elem_num = 1
iteration = 300000

In [ ]:
# load the dataset
dataframe = read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv', usecols=[1], engine='python', skipfooter=3)
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
#dataset = scaler.fit_transform(dataframe.values.astype('float32'))
dataset = dataframe.values.astype('float32')

data=np.zeros((batch_num,step_num,1))
for b in range(batch_num):
    data[b]=dataset[b:b+step_num]
    
# (batch_num, 8, 1)
 
print(data.shape)
    

In [ ]:
# placeholder list
p_input = tf.placeholder(tf.float32, shape=(batch_num, step_num, elem_num))
p_inputs = [tf.squeeze(t, [1]) for t in tf.split(p_input, step_num, 1)]

cell = tf.nn.rnn_cell.LSTMCell(hidden_num, use_peepholes=True)
ae = LSTMAutoencoder(hidden_num, p_inputs, cell=cell, decode_without_input=True)

In [ ]:
# Encapsuler ça dans une fonction au format Train

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(iteration):
        iteration_data = np.copy(data)
        np.random.shuffle(iteration_data)
        (loss_val, _) = sess.run([ae.loss, ae.train], {p_input: iteration_data})
        if i%(iteration/20) == 0:
            print('iter %d:' % (i), loss_val)

    (input_, output_) = sess.run([ae.input_, ae.output_], {p_input: data})
    print('train result :')
    print('input : ', input_[0, :, 0])
    print('output :', output_[0, :, 0])
    print('diff :  ', output_[0, :, 0]-input_[0, :, 0])

In [ ]:
import matplotlib.pyplot as plt
plt.plot(input_[0, :, 0])
plt.plot(output_[0, :, 0])
plt.show()

In [ ]:
z=-10
import matplotlib.pyplot as plt
plt.plot(input_[z, :, 0])
plt.plot(output_[z, :, 0])
plt.show()